In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp
import optax
import equinox as eqx

from flex import FuzzyVariable
from flex.visualize import plot_mfs
from flex.fiss import TSK

In [ ]:
f = lambda x, y: np.sin(x) * np.cos(y)

seed = 12345
np.random.seed(seed)

N = 20000
batch_size = 128
lr = 1e-3
epochs = 50

x = np.random.uniform(-np.pi, np.pi, size=(N, 1))
y = np.random.uniform(-np.pi, np.pi, size=(N, 1))
X = np.hstack([x, y])
Y = f(x, y)

_X = (X + np.pi) / (2.0*np.pi)

val_frac = 0.2
idx = np.random.permutation(N)
n_val = int(N * val_frac)
val_idx, train_idx = idx[:n_val], idx[n_val:]

x_train = jnp.asarray(_X[train_idx])
y_train = jnp.asarray(Y)
